In [25]:
import Pkg; using Pkg
using LinearAlgebra
using Plots
using DataFrames
using CSV
using Statistics
using Random
using StatsBase


In [26]:
# 1. dataShape: to get shape of the data
function dataShape(C)
    return size(C)
end

# 2. dataType: gives data type of each column in the dataset
function dataType(C, output = "dict")
    if output == "dict"
        column_info = Dict(names(C) .=> [eltype(C[!, col]) for col in names(C)])

    else
        column_info = []
        for i in 1:size(C, 2)
            push!(column_info, (names(C)[i], eltype(C[!, i])))
        end
    end
    
    return column_info
end

# 3. count_missing(col) : counts number of missing data in a given col (Column).
function count_missing(C, col)
    return sum(ismissing.(C[:, col]))
end

# 4. dataMissingPercentage(): finds missing percentage of each column.
function dataMissingPercentage(C)
    length_C = size(C, 1)
    missing_percentage = []
    for i in 1:size(C, 2)
        push!(missing_percentage, (names(C)[i], sum(ismissing.(C[:, i])) / length_C))
    end
    return missing_percentage
end

# 5. deleteColumns(threshold) : delete all the columns which have missing percent less than given threshold.
function deleteColumns(C,threshold)
   length_C = size(C, 1)
          
    return C[:, [count_missing(C, i) / length_C < threshold for i in 1:size(C, 2)]]
end

# 5.1 deleteColumns(threshold,deleteColumns) : delete the columns in deleteColumns.
function deleteSelectedColumns(C,deleteColumns)
    return select(C, Not(deleteColumns))
end

# 6. calculateCorrelation() : creates correlation matrix between columns.
function calculateCorrelation(C)
    return cor(C)
end

# 7. displayCorrelation() : dislpay correlation using heatmap.
function displayCorrelation(C)
    
    return heatmap(correlation_mat, colormap=:bam , axis=(title="Mapa de calor de matriz de correlacion",))
end

# 8. removeOutliersIQR() : using interquartile range delete all the outliers from the numerical columns.
function removeOutliersIQR(C, column)
   
    data = C[:,column]  # Ejemplo de datos
    rango_iqr = iqr(C[:,column])
    # Calcular Q1 y Q3
    Q1 = quantile(data, 0.25)
    Q3 = quantile(data, 0.75)

    println("El rango intercuartílico es: ", rango_iqr)
    println("El Q1 es: ", Q1)
    println("El Q3 es: ", Q3)

    # Definir límites para outliers
    limite_inferior = Q1 - 1.5 * rango_iqr
    limite_superior = Q3 + 1.5 * rango_iqr

    # Encontrar índices de outliers
    indices_outliers = findall(x -> x < limite_inferior || x > limite_superior, data)
    println("Número de outliers encontrados: ", length(indices_outliers))

    # Eliminar filas con outliers
    data_outliers = C[setdiff(1:nrow(C), indices_outliers), :]  
    return data_outliers

end

# 9. deleteRow(column): for a given column delete all the null data points.
function deleteRow(C, column)
    return C[ismissing.(C[:, column]), :]
end

# 10. filterColumnsByCorrelation(target,threshold, realtion) : delete all the columns on the basis of given threshold for a target column and relation .
function filterColumnsByCorrelation(C, target, threshold, relation)
# Obtener el índice de la columna target (T_degC)
target_index = findfirst(isequal(target), names(c_working))
|
indices_mayor_correlacion = findall(x -> x > threshold, correlation_mat[:, target_index])
c_working_2 =select(c_working, indices_mayor_correlacion)


    return select(c_working, indices_mayor_correlacion)
end

# 11. describeData() : it is used to describe data by giving following for each column/
function describeData(C)
    println("El tamaño del data set es: ", size(C))
    return describe(C)
end
 

describeData (generic function with 1 method)

In [27]:
println("Este programa explora los datos del conjunto: bottle ")
C = CSV.read("../dat/bottle.csv", DataFrame);
size(C)

Este programa explora los datos del conjunto: bottle 


(864863, 74)

In [28]:
#obtener conjunto de datos para pruebas del código
# Definir el porcentaje deseado
porcentaje = 0.2  # 20%

# Calcular el número de índices a seleccionar
num_indices = Int(round(porcentaje * nrow(C)))

# Obtener índices aleatorios
indices_aleatorios = sample(1:nrow(C), num_indices, replace=false)

# Imprimir los índices seleccionados
#println("Índices aleatorios seleccionados")
#println(indices_aleatorios)
@show length(indices_aleatorios)
C = C[indices_aleatorios, :]
size(C)


length(indices_aleatorios) = 172973


(172973, 74)

In [29]:
# borrando columnas con más del 65% de datos faltantes
c_working = deleteColumns(C, 0.65)
size(c_working)

(172973, 44)

In [30]:
"Limpiando columnas"
# Obtener las columnas de tipo String en c_working
columnas_string = names(c_working)[eltype.(eachcol(c_working)) .<: Union{String, Missing, String15}]
println("Colum  nas de tipo String en c_working: ", columnas_string)
c_working= deleteSelectedColumns(c_working, columnas_string)

# Reemplazar valores missing con la media de cada columna o valor 0
for col in names(c_working)
    # Obtener los valores no-missing de la columna
    valores_no_missing = skipmissing(c_working[!, col])
    media = 0

    # Reemplazar los missing con la media
    c_working[!, col] = coalesce.(c_working[!, col], media)
end

println("Se han reemplazado los valores missing con la media de cada columna")
describeData(c_working)


Colum  nas de tipo String en c_working: ["Sta_ID", "Depth_ID"]
Se han reemplazado los valores missing con la media de cada columna
El tamaño del data set es: (172973, 42)


42×7 DataFrame
 Row │ variable     mean           min        median       max         nmissin ⋯
     │ Symbol       Float64        Real       Float64      Real        Int64   ⋯
─────┼──────────────────────────────────────────────────────────────────────────
   1 │ Cst_Cnt      17144.2         1          16874.0      34403              ⋯
   2 │ Btl_Cnt          4.32562e5   4         432778.0     864852
   3 │ Depthm         226.801       0            125.0       5062
   4 │ T_degC          10.6678      0.0           10.0         30.46
   5 │ Salnty          32.0156      0.0           33.81        36.615          ⋯
   6 │ O2ml_L           2.72947    -0.01           2.46         9.98
   7 │ STheta          24.2701      0.0           25.875      250.784
   8 │ O2Sat           43.7245     -0.1           34.0        191.5
  ⋮  │      ⋮             ⋮            ⋮           ⋮           ⋮          ⋮    ⋱
  36 │ R_O2             2.72949    -0.01           2.46         9.98           ⋯
  37 │ R_O2Sat         44.1241     -0.1           34.8        191.5
  38 │ R_SIO3          10.9361      0.0            0.0        186.0
  39 │ R_PO4            0.753369    0.0            0.0          4.96
  40 │ R_NO3            6.78222     0.0            0.0         87.0            ⋯
  41 │ R_NO2            0.0163404   0.0            0.0          4.21
  42 │ R_PRES         228.365       0            126.0       5156
                                                   2 columns and 27 rows omitted

In [31]:
correlation_mat = calculateCorrelation(Matrix(c_working))

42×42 Matrix{Float64}:
  1.0         0.999343   -0.162942   …   0.538392    0.249556   -0.162672
  0.999343    1.0        -0.162488       0.532782    0.248498   -0.162211
 -0.162942   -0.162488    1.0            0.130237   -0.117877    0.999994
  0.100948    0.101983   -0.639148      -0.311244    0.085812   -0.637877
  0.220122    0.21918    -0.0428262      0.111183    0.0346684  -0.0431187
  0.191431    0.190978   -0.440991   …  -0.153701    0.218362   -0.439707
  0.205034    0.203977    0.0433273      0.160823    0.030035    0.0428419
  0.226372    0.225655   -0.435973      -0.161805    0.209095   -0.434783
  0.237205    0.236394   -0.422361      -0.133079    0.222196   -0.421172
 -0.103375   -0.104073    0.0810368     -0.0389644  -0.0638432   0.0808997
  ⋮                                  ⋱               ⋮          
  0.214387    0.213757   -0.580045      -0.258352    0.113069   -0.578854
 -0.0690972  -0.0696007   0.795658       0.265986   -0.133931    0.794351
  0.191446    0.19099

In [32]:
#filt delete all the columns on the basis of given threshold for a target column and relation .
target = "T_degC"
threshold = .60
# Obtener el índice de la columna target (T_degC)
target_index = findfirst(isequal(target), names(c_working))
indices_mayor_correlacion = findall(x -> x > threshold, correlation_mat[:, target_index])
c_working=select(c_working, indices_mayor_correlacion)
dataType(c_working, "list")


6-element Vector{Any}:
 ("T_degC", Real)
 ("O2Sat", Real)
 ("R_TEMP", Real)
 ("R_POTEMP", Real)
 ("R_SVA", Real)
 ("R_O2Sat", Real)

In [14]:
# Obtener todas las columnas excepto T_degC
otras_columnas = filter(x -> x != "T_degC", names(c_working))

# Crear un plot para cada columna vs T_degC
plots = []
for columna in otras_columnas
    p = scatter( c_working.T_degC,c_working[!, columna], 
               xlabel="T_degC", 
               ylabel=columna,
               title="$columna vs T_degC",
               legend=false,
               alpha=0.5)
    push!(plots, p)
end



In [15]:
# Mostrar todos los plots en una cuadrícula
x = plot(plots..., layout=(3,3), size=(1200,1000))
savefig(x, "../fig/plots.png") 



"/home/cecilia/Desktop/Ejercicios/Exercise2/fig/plots.png"

In [33]:
# quitar outliers iqr
#using StatsBase       # Importa la biblioteca

c_working = removeOutliersIQR(c_working, "T_degC")

c_working = removeOutliersIQR(c_working, "O2Sat")
c_working = removeOutliersIQR(c_working, "R_SVA")
println("Dimensiones después de eliminar outliers: ", size(c_working))





El rango intercuartílico es: 6.26
El Q1 es: 7.58
El Q3 es: 13.84
Número de outliers encontrados: 738
El rango intercuartílico es: 84.3
El Q1 es: 3.8
El Q3 es: 88.1
Número de outliers encontrados: 0
El rango intercuartílico es: 160.10000000000002
El Q1 es: 133.2
El Q3 es: 293.3
Número de outliers encontrados: 3
Dimensiones después de eliminar outliers: (172232, 6)


In [18]:
function find_best_fit(xvals,yvals)
    meanx = mean(xvals)
    meany = mean(yvals)
    stdx = std(xvals)
    stdy = std(yvals)
    r = cor(xvals,yvals)
    a = r*stdy/stdx
    b = meany - a*meanx
    return a,b
end

function eliminar_vacios_(arr)
    # Encontrar índices de elementos no vacíos
    indices_validos = findall(!ismissing, arr)
    
    # Encontrar índices de elementos vacíos
    indices_vacios = findall(ismissing, arr)
    
    # Crear nuevo array sin valores vacíos
    arr_limpio = arr[indices_validos]
    
    return arr_limpio, indices_validos
end

eliminar_vacios_ (generic function with 1 method)

In [45]:
xvals = c_working[!, "T_degC"]
yvals = c_working[!, "R_SVA"]

yvals_, indices_validos = eliminar_vacios_(yvals)
#limpiar x
xvals_ = xvals[indices_validos]



@show size(xvals_)
@show size(yvals_)



size(xvals_) = (172232,)
size(yvals_) = (172232,)


(172232,)

In [46]:
xvals_ = coalesce.(xvals_, 0)   
yvals_ = coalesce.(yvals_, 0)

172232-element Vector{Real}:
 180.8
 267.5
 312.8
 182.7
 126.1
 133.9
 231.1
 146.5
 312.0
 272.4
   ⋮
  78.0
  93.8
 122.9
 405.6
 265.8
 147.6
 117.4
 231.8
 169.8

In [47]:
a,b = find_best_fit(xvals_,yvals_)
ynew = a .* xvals_ .+ b




172232-element Vector{Float64}:
 172.57638806938897
 231.2981052538314
 267.4506215275273
 181.14432545217238
 129.73670115547185
 131.19951973302022
 209.14685250809868
 137.0507940432138
 286.6762371181633
 255.5390988246333
   ⋮
  66.4175541558773
  90.03162547915844
 135.17002730065158
 384.68508181390536
 236.10450915149042
 176.96484380203412
 101.52520001703864
 208.31095617807102
 174.03920664693734

In [48]:
x = scatter(xvals_, ynew,
    markersize=2, 
    label="Ajuste lineal",
    color=:red)

    scatter!(xvals_, yvals_,
    markersize=1,
    label="Datos originales", 
    color=:blue,

    xlabel="T_degC", 
    ylabel="R_SVA",
    title="R_SVA vs T_degC"
    )
savefig(x, "../fig/regresionLineal.png") 

"/home/cecilia/Desktop/Ejercicios/Exercise2/fig/regresionLineal.png"

In [49]:
xvals = c_working[!, "T_degC"]
yvals = c_working[!, "O2Sat"]

yvals_, indices_validos = eliminar_vacios_(yvals)
#limpiar x
xvals_ = xvals[indices_validos]

a,b = find_best_fit(xvals_,yvals_)
ynew = a .* xvals_ .+ b

172232-element Vector{Float64}:
 33.70466415930756
 50.74390606910264
 61.23425784986616
 36.19081689347695
 21.27390048846061
 21.698365589416362
 44.31629168320129
 23.396225993239362
 66.81294203385602
 57.777899170655054
  ⋮
  2.9006254042331747
  9.752704891090275
 22.8504851491534
 95.25210379789121
 52.1385771151001
 34.978059462174805
 13.087787827171162
 44.07374019694087
 34.12912926026331

In [50]:
x = scatter(xvals_, ynew,
    markersize=2, 
    label="Ajuste lineal",
    color=:red)

    scatter!(xvals_, yvals_,
    markersize=1,
    label="Datos originales", 
    color=:blue,
    xlabel="T_degC", 
    ylabel="O2Sat",
    title="O2Sat vs T_degC")

savefig(x, "../fig/regresionLineal_O2Sat.png") 

"/home/cecilia/Desktop/Ejercicios/Exercise2/fig/regresionLineal_O2Sat.png"